In [1]:
# Install correct dependencies
!pip install tables
!pip install numpy SimpleITK tqdm xlrd pandas progressbar matplotlib nilearn sklearn
!pip install git+https://www.github.com/farizrahman4u/keras-contrib.git
!pip install tensorflow==1.11.0
!pip install keras==2.2.4

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
  Using cached https://files.pythonhosted.org/packages/b3/b4/a710fad882736dcd42d5e4ac49c953be002408d20981dc2b8a2bc6837729/tables-3.5.2-cp27-cp27m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a8/49/b27f05c86de31e8db391bf5dee750bfc16d8ffd50b97f72c19b8a2d6211a/numexpr-2.7.1-cp27-cp27m-macosx_10_6_intel.whl
  Using cached https://files.pythonhosted.org/packages/05/d2/f94e68be6b17f46d2c353564da56e6fb89ef09faeeff3313a046cb810ca9/mock-3.0.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/09/96/84cf406fe7d589f3dba9fc0f737e65985a3526c6d8c783f02d4b5a10825d/numpy-1.16.6-cp27-cp27m-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packa

  Stored in directory: /private/var/folders/xn/yfwfzxps1l70gvxtwg_7l4zh0000gn/T/pip-ephem-wheel-cache-jJRLY1/wheels/1f/8e/ac/fb1cca9d92276d64365b204e82a9a5bec1f24a20aca28fdbec
Successfully built keras-contrib
matplotlib 1.3.1 requires nose, which is not installed.
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/Library/Python/2.7/site-packages/PyYAML-5.3.1.dist-info'
Consider using the `--user` option or check the permissions.

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
  Using cached https://files.pythonhosted.org/packages/c1/d4/623b04520c344bd4b861efe89c83cef1f2d0d24d6791704d15920a1abd4a/tensorflow-1.11.0-cp27-cp27m-macosx_10_11_x86_64.whl
    100% |████████████████████████████████| 1.3MB 1.3MB/s ta 0:00:01
    100% |████████████████████████████████| 3.0MB 2.3MB/s ta 0:00:01
   

matplotlib 1.3.1 requires nose, which is not installed.
  Found existing installation: numpy 1.8.0rc1
Cannot uninstall 'numpy'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [2]:
import tensorflow
print(tensorflow.__version__)
# If this does not print 1.11.0 restart the runtime after executing the above. (Ctrl+M)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)           # we mount the google drive at /content/drive

In [ ]:
import zipfile  # For faster extraction
zfile = zipfile.ZipFile("/content/drive/Shared drives/Brats/brats19.zip")
zfile.extractall()

In [ ]:
# Get in there
%cd brats19/demo_task2

In [ ]:
import sys
sys.path.append('..')
import preprocess
import train_model
import os
import tensorflow as tf
import keras.backend as K

In [ ]:
from tqdm import tqdm
import pandas as pd
import nibabel as nib
import numpy as np
import os
from keras import models
from keras import layers
import matplotlib.pyplot as plt
import pdb
from keras.layers import Flatten,Dense
from keras.optimizers import RMSprop,Adam
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping
from keras.models import load_model
import keras.backend as K
from dev_tools.my_tools import my_makedirs

config = dict()
config['initilizer'] = True
config['l2'] = True
config['model_file'] = 'os_model.h5'
config['feature_indices'] = [4,5,6,10,11,12,13]

config['training_target_csv'] = os.path.abspath('content/drive/Shared drives/Brats/training_gtr_only.csv')
config['training_source_csv'] = os.path.abspath('content/drive/Shared drives/Brats/survival_data.csv')
config['training_npz'] = os.path.abspath('content/drive/Shared drives/Brats/training_mine_gtr_only.npz')

config['val_target_csv'] = os.path.abspath('content/drive/Shared drives/Brats/val_gtr_only.csv')
config['val_source_csv'] = os.path.abspath('content/drive/Shared drives/Brats/survival_evaluation.csv')
config['val_npz'] = os.path.abspath('content/drive/Shared drives/Brats/val_mine_gtr_only.npz')

# implement leave-one-out cross-validation
def LOOCV(npz_file = '../data/os_data/training_mine_gtr_only.npz', feature_index = [4,5,6,10,11,12,13]):
  xy_zip = np.load(npz_file)
  train_data = xy_zip['arr_0']
  train_targets = xy_zip['arr_1']
  num_epochs = 800

  if feature_index:
    train_data = train_data[:, feature_index]

  for i in range(len(train_data)):
    trn_data = np.delete(train_data, i, axis = 0)
    trn_targets = np.delete(train_targets, i, axis = 0)
    val_data = train_data[i]
    val_targets = val_data[i]
    model = build_model((train_data.shape[1],))
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, min_lr=1e-8)
    history = model.fit(trn_data, 
                        trn_targets,
                        validation_data = (val_data, val_targets), 
                        epochs = num_epochs,
                        batch_size = 5,
                        verbose = 0,
                        callbacks = [reduce_lr])
    print(model.evaluate(val_data, val_targets))
    val_lms = history.history['val_lms_acc']
    train_lms = history.history['lms_acc']
    mae_history = history.history['val_mean_absolute_error']
    val_loss = history.history['val_loss']
    train_loss = history.history['loss']
    train_mae = history.history['mean_absolute_error']

    plt.figure()
    plt.plot(range(1, len(mae_history) + 1), val_loss,label='val_loss')
    plt.plot(range(1, len(mae_history) + 1), train_loss,label='train_loss')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('mse')
    plt.show()

# implement k-fold cross-validation
def cross_validation(npz_file = '../data/os_data/training_mine_gtr_only.npz', feature_index = [4,5,6,10,11,12,13]):
  xy_zip = np.load(npz_file)
  train_data = xy_zip['arr_0']
  train_targets = xy_zip['arr_1']

  if feature_index:
    train_data = train_data[:, feature_index]
  
  k = 4
  num_epochs = 800
  num_per_fold = len(train_data) / k

  for fold in range(k):
    
    start_idx = int(fold * num_per_fold)
    end_idx = int((fold + 1) * num_per_fold)

    trn_data = np.delete(train_data, slice(start_idx, end_idx), axis = 0)
    trn_targets = np.delete(train_targets, slice(start_idx, end_idx), axis = 0)
    val_data = train_data[start_idx : end_idx]
    val_targets = train_targets[start_idx : end_idx]

    model = build_model((train_data.shape[1],))
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, min_lr=1e-8)
    history = model.fit(trn_data, 
                        trn_targets,
                        validation_data = (val_data, val_targets), 
                        epochs = num_epochs,
                        batch_size = 5,
                        verbose = 0,
                        callbacks = [reduce_lr])
    print(model.evaluate(val_data, val_targets))
    val_lms = history.history['val_lms_acc']
    train_lms = history.history['lms_acc']
    mae_history = history.history['val_mean_absolute_error']
    val_loss = history.history['val_loss']
    train_loss = history.history['loss']
    train_mae = history.history['mean_absolute_error']

    plt.figure()
    plt.plot(range(1, len(mae_history) + 1), val_loss,label='val_loss')
    plt.plot(range(1, len(mae_history) + 1), train_loss,label='train_loss')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('mse')
    plt.show()

# implement monte carlo 
def monte_carlo(npz_file = '../data/os_data/training_mine_gtr_only.npz', feature_index = [4,5,6,10,11,12,13]):
  xy_zip = np.load(npz_file)
  train_data = xy_zip['arr_0']
  train_targets = xy_zip['arr_1']
  num_epochs = 800

  if feature_index:
    train_data = train_data[:, feature_index]

  n_mc_iters = 20
  n_per_mc_iter = 20

  for it in range(n_mc_iters):
    rand_points = np.random.choice(len(train_data), n_per_mc_iter)

    trn_data = np.delete(train_data, rand_points, axis = 0)
    trn_targets = np.delete(train_targets, rand_points, axis = 0)
    val_data = train_data[rand_points]
    val_targets = train_targets[rand_points]

    model = build_model((train_data.shape[1],))
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, min_lr=1e-8)
    history = model.fit(trn_data, 
                        trn_targets,
                        validation_data = (val_data, val_targets), 
                        epochs = num_epochs,
                        batch_size = 5,
                        verbose = 0,
                        callbacks = [reduce_lr])
    print(model.evaluate(val_data, val_targets))
    val_lms = history.history['val_lms_acc']
    train_lms = history.history['lms_acc']
    mae_history = history.history['val_mean_absolute_error']
    val_loss = history.history['val_loss']
    train_loss = history.history['loss']
    train_mae = history.history['mean_absolute_error']

    plt.figure()
    plt.plot(range(1, len(mae_history) + 1), val_loss,label='val_loss')
    plt.plot(range(1, len(mae_history) + 1), train_loss,label='train_loss')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('mse')
    plt.show()

def lms_acc(y_true, y_pred):
    '''
    long-mid-short survivor accuracy
    '''
#     pdb.set_trace()
    y_pred = K.reshape(y_pred,(-1,))
    y_true = K.reshape(y_true,(-1,))
    long_survivors = K.sum(K.cast(y_pred > 15*30,'int32') * K.cast(y_true > 15*30, 'int32'))
    short_survivors = K.sum(K.cast(y_pred < 10*30,'int32') * K.cast(y_true < 10*30, 'int32'))
    mid_survivors = K.sum(K.cast(y_pred < 15*30,'int32') * K.cast(y_pred > 10*30,'int32')
                        * K.cast(y_true < 15*30,'int32') * K.cast(y_true > 10*30,'int32')) 
    return (long_survivors + short_survivors + mid_survivors)/K.shape(y_true)[0]

def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=input_shape))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
#     model.compile(optimizer='rmsprop', loss='mse', metrics=['mae','mse','acc'])
#     model.compile(optimizer=RMSprop(lr=0.001), loss='mse', metrics=['mae', 'acc'])
    model.compile(optimizer=Adam(lr=1e-4), loss='mse', metrics=['mae', 'mse','acc',lms_acc])
    return model
    

def final_train(epochs=800, npz_file='../data/os_data/training_mine_gtr_only.npz',feature_index=[4,5,6,10,11,12,13]):
    xy_zip = np.load(npz_file)
    train_data = xy_zip['arr_0']
    train_targets = xy_zip['arr_1']
    
    if feature_index:
        train_data = train_data[:,feature_index]
        
    val_data = train_data[25: 50]
    val_targets = train_targets[25:50]
    
    mean = train_data.mean(axis=0)
    train_data -= mean
    std = train_data.std(axis=0)
    train_data /= std
    
    model = build_model((train_data.shape[1],))
    
    callbacks_list=[]
    callbacks_list.append(ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=10, min_lr=1e-8))
    callbacks_list.append(ModelCheckpoint(config['model_file'], save_best_only=True))
    callbacks_list.append(EarlyStopping(patience=50))
    history = model.fit(train_data, 
                        train_targets,
                        validation_data=(val_data,val_targets),
                        epochs=epochs, 
                        batch_size=5, 
                        verbose=0,
                        callbacks=callbacks_list
                        )

    print(model.evaluate(val_data,val_targets))
    print('predicted values:', model.predict(val_data).reshape(-1,))
    print('targeted values:', val_targets)
#     pdb.set_trace()
    
    val_lms = history.history['val_lms_acc']
    train_lms = history.history['lms_acc']
    mae_history = history.history['val_mean_absolute_error']
    val_loss = history.history['val_loss']
    train_loss = history.history['loss']
    train_mae = history.history['mean_absolute_error']

    plt.figure()
    plt.plot(range(1, len(mae_history) + 1), mae_history,label='val_mae')
    plt.plot(range(1, len(mae_history) + 1), train_mae,label='train_mae')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('MAE')
    plt.show()

    plt.figure()
    plt.plot(range(1, len(mae_history) + 1), val_loss,label='val_loss')
    plt.plot(range(1, len(mae_history) + 1), train_loss,label='train_loss')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('mse')
    plt.show()
    
    plt.figure()
    plt.plot(range(1, len(mae_history) + 1), val_lms,label='val_lms')
    plt.plot(range(1, len(mae_history) + 1), train_lms,label='train_lms')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('lms acc')
    plt.show()
    return model

def evaluate(npz_file='../data/os_data/training_mine_gtr_only.npz',feature_index=[4,5,6,10,11,12,13]):
#     pdb.set_trace()
    xy_zip = np.load(npz_file)
    train_data = xy_zip['arr_0']
    train_targets = xy_zip['arr_1']
    if feature_index:
        train_data = train_data[:,feature_index]
    mean = train_data.mean(axis=0)
    train_data -= mean
    std = train_data.std(axis=0)
    train_data /= std
    val_data = train_data[25: 50]
    val_targets = train_targets[25:50]
    
    model = load_model(config['model_file'],custom_objects={'lms_acc':lms_acc})
    predicted = np.reshape(model.predict(val_data),(-1,))
    
    print('predicted survival days:',predicted)
    print('val_targets:',val_targets)
    print(model.evaluate(val_data,val_targets))
    return predicted, val_targets

def predict():
#     pdb.set_trace()
    xy_zip = np.load(config['training_npz'])
    train_data = xy_zip['arr_0']
    train_data = train_data[:,config['feature_indices']]
    mean = train_data.mean(axis=0)
    train_data -= mean
    std = train_data.std(axis=0)
    train_data /= std
    
    xy_zip = np.load(config['val_npz'])
    val_data = xy_zip['arr_0']
    val_data = val_data[:,config['feature_indices']]
    val_data -= mean
    val_data /= std
    
    model = load_model(config['model_file'],custom_objects={'lms_acc':lms_acc})
    
    train_predicted = model.predict(train_data).reshape(-1,)
    val_predicted = model.predict(val_data).reshape(-1,)
    
#     print(train_predicted)
#     print(val_predicted)
    train_predicted = np.round(train_predicted).astype(int)
    val_predicted = np.round(val_predicted).astype(int)
    
    my_makedirs('saves')
    
    df = pd.read_csv(config['training_target_csv'])
    id_list = list(df['BraTS19ID'])
    os_list = list(train_predicted)
    data_to_save = pd.DataFrame({'id':id_list,'os':os_list})
    data_to_save.to_csv(os.path.join('saves','training_to_upload.csv'), index=False, sep=',',header=False)
    

    # NOTE --- Using BraTS18ID for validation data.
    #   We have the features extracted from the 2019 training data, but had
    #   to generate the validation results ourselves on the 2018 data,
    #   using the network pretrained on the '19 data. 
    df = pd.read_csv(config['val_target_csv'])
    id_list = list(df['BraTS18ID'])
    os_list = list(val_predicted)
    data_to_save = pd.DataFrame({'id':id_list,'os':os_list})
    data_to_save.to_csv(os.path.join('saves','val_to_upload.csv'), index=False, sep=',',header=False)
    return

if __name__ == '__main__':
    #cross_validation()
    #print("Cross validation")

    monte_carlo()
    print("done")
    final_train()
    evaluate()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import zipfile  # For faster extraction

# train_path = "/content/drive/Shared drives/Brats/MICCAI_BraTS_2018_Data_Training.zip"
# valid_path = "/content/drive/Shared drives/Brats/MICCAI_BraTS_2018_Data_Validation.zip"

# # Create data directory
# %mkdir data 
# %cd data 

# # Unzip training data
# %mkdir original
# %cd original
# zfile = zipfile.ZipFile(train_path)
# zfile.extractall()
# #%mv survival_data.csv ../
# %cd ../

# # Unzip validation data
# %mkdir val
# %cd val
# %mkdir val
# %cd val
# zfile = zipfile.ZipFile(valid_path)
# zfile.extractall()